<img src = "https://8weeksqlchallenge.com/images/case-study-designs/2.png">

## Introduction

Did you know that over 115 million kilograms of pizza is consumed daily worldwide??? (Well according to Wikipedia anyway…)

Danny was scrolling through his Instagram feed when something really caught his eye - “80s Retro Styling and Pizza Is The Future!”

Danny was sold on the idea, but he knew that pizza alone was not going to help him get seed funding to expand his new Pizza Empire - so he had one more genius idea to combine with it - he was going to Uberize it - and so Pizza Runner was launched!

## Problem Statement

Danny started by recruiting “runners” to deliver fresh pizza from Pizza Runner Headquarters (otherwise known as Danny’s house) and also maxed out his credit card to pay freelance developers to build a mobile app to accept orders from customers.


## Entity Relationship Diagram

<img src = "https://user-images.githubusercontent.com/81607668/127271531-0b4da8c7-8b24-4a14-9093-0795c4fa037e.png">

In [1]:
import sqlite3
import pandas as pd

In [2]:
database = '../data/PizzaRunner.sqlite'

In [3]:
connection = sqlite3.connect(database)

# Case Study Questions

---

## A. Pizza Metrics

---

### 1. How many pizzas were ordered?

In [4]:
pd.read_sql("""

            SELECT COUNT((order_id)) AS "No. of Pizza's ordered"
            FROM customer_orders
            
            """,connection)

,No. of Pizza's ordered
0,14


### 2. How many unique customer orders were made?

In [5]:
pd.read_sql("""

            SELECT customer_id AS Customer, COUNT(DISTINCT(order_id)) AS "Unique Orders"
            FROM customer_orders
            GROUP BY Customer
            
            """,connection)

,Customer,Unique Orders
0,101,3
1,102,2
2,103,2
3,104,2
4,105,1


### 3. How many successful orders were delivered by each runner?

In [6]:
pd.read_sql("""

            SELECT COUNT(order_id) AS "Successful Orders"
            FROM runner_orders
            WHERE distance != 0
            
            """,connection)

,Successful Orders
0,8


### 4. How many of each type of pizza was delivered?

In [7]:
pd.read_sql("""

            SELECT c.pizza_id AS "Pizza ID", p.pizza_name AS Pizza, COUNT(c.pizza_id) AS "No.of Delivered Pizza's"
            FROM customer_orders AS c
            INNER JOIN runner_orders AS r
                ON c.order_id = r.order_id
            INNER JOIN pizza_names AS p
                ON c.pizza_id = p.pizza_id
            WHERE distance != 0
            GROUP BY "Pizza ID"
            
            """,connection)

,Pizza ID,Pizza,No.of Delivered Pizza's
0,1,Meatlovers,9
1,2,Vegetarian,3


### 5. How many Vegetarian and Meatlovers were ordered by each customer?

In [8]:
pd.read_sql("""

            SELECT c.customer_id AS Customer, p.pizza_name AS Pizza, COUNT(p.pizza_name) AS "No. of Orders"
            FROM customer_orders AS c
            INNER JOIN pizza_names AS p
                ON c.pizza_id= p.pizza_id
            GROUP BY Customer, Pizza
            ORDER BY Customer
            
            """,connection)

,Customer,Pizza,No. of Orders
0,101,Meatlovers,2
1,101,Vegetarian,1
2,102,Meatlovers,2
3,102,Vegetarian,1
4,103,Meatlovers,3
5,103,Vegetarian,1
6,104,Meatlovers,3
7,105,Vegetarian,1


### 6. What was the maximum number of pizzas delivered in a single order?

In [9]:
pd.read_sql("""
            
            WITH pizza_per_order AS
            (
            SELECT c.order_id, COUNT(c.pizza_id) AS pizza_per_order
            FROM customer_orders AS c
            JOIN runner_orders AS r
                ON c.order_id = r.order_id
            WHERE r.distance != 0
            GROUP BY c.order_id
            )

            SELECT MAX(pizza_per_order) AS "Pizza Count"
            FROM pizza_per_order

            
            """,connection)

,Pizza Count
0,3


### 7.  For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [10]:
pd.read_sql("""

            SELECT 
              c.customer_id AS Customer,
              SUM(
                CASE WHEN c.exclusions != ' ' OR c.extras != ' ' THEN 1
                ELSE 0
                END) AS at_least_1_change,
              SUM(
                CASE WHEN c.exclusions = ' ' AND c.extras = ' ' THEN 1 
                ELSE 0
                END) AS no_change

            FROM customer_orders AS c
            JOIN runner_orders AS r
              ON c.order_id = r.order_id

            WHERE r.distance != 0
            GROUP BY Customer
            ORDER BY Customer;

            """,connection)

,Customer,at_least_1_change,no_change
0,101,0,0
1,102,0,0
2,103,3,0
3,104,2,0
4,105,1,0


### 8. How many pizzas were delivered that had both exclusions and extras?

In [11]:
pd.read_sql("""
            
            SELECT  
              SUM(
                CASE WHEN exclusions IS NOT NULL AND extras IS NOT NULL THEN 1
                ELSE 0
                END) AS pizza_with_exclusions_and_extras
            FROM customer_orders AS c
            JOIN runner_orders AS r
              ON c.order_id = r.order_id
            WHERE r.distance >= 1 
              AND exclusions != ' ' 
              AND extras != ' ';
            
            """,connection)

,pizza_with_exclusions_and_extras
0,1


### 9. What was the total volume of pizzas ordered for each hour of the day?

In [12]:
pd.read_sql("""

            SELECT 
              strftime('%H', order_time) AS "Hour of day", 
              COUNT(order_id) AS "Pizza Count"
            FROM customer_orders
            GROUP BY "Hour of day";         
            
            """,connection)

,Hour of day,Pizza Count
0,11,1
1,13,3
2,18,3
3,19,1
4,21,3
5,23,3


### 10. What was the volume of orders for each day of the week?

In [13]:
pd.read_sql("""

        SELECT  

        CASE CAST(strftime('%w', order_time) as integer)
          WHEN 0 THEN 'Sunday'
          WHEN 1 THEN 'Monday'
          WHEN 2 THEN 'Tuesday'
          WHEN 3 THEN 'Wednesday'
          WHEN 4 THEN 'Thursday'
          WHEN 5 THEN 'Friday'
          ELSE 'Saturday' 
        END AS "Week Day",

        COUNT(order_id) AS "Total Pizzas"

        FROM customer_orders
        GROUP BY "Week Day"
        ORDER BY "Total Pizzas" DESC

""", connection
)

,Week Day,Total Pizzas
0,Wednesday,5
1,Saturday,5
2,Thursday,3
3,Friday,1
